# Point-to-Mesh Similarity Analysis

This notebook demonstrates how to:
1. Load two meshes
2. Specify a particular point on the source mesh
3. Color-map the target mesh based on similarity to that point

The analysis uses diffusion features + DINO features to compute semantic similarity between mesh vertices.

## 1. Import Dependencies

In [1]:
import torch
from diff3f import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_dino
from functional_map import compute_surface_map
import importlib
import meshplot as mp
from point_to_mesh_similarity import run_point_similarity_analysis, point_similarity_colormap, visualize_point_similarity, run_multi_point_correspondence_analysis

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/home/zixiliu/miniforge3/envs/diff3f/lib/python3.10/site-packages/accelerate/utils/torch_xla.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Using device: cuda


## 2. Initialize Models

This step loads the diffusion pipeline and DINO model for feature extraction.

In [2]:
print("Initializing diffusion pipeline...")
pipe = init_pipe(device)

print("Initializing DINO model...")
dino_model = init_dino(device)

print("Models initialized successfully!")

Initializing diffusion pipeline...


/home/zixiliu/miniforge3/envs/diff3f/lib/python3.10/site-packages/diffusers/configuration_utils.py:239: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'unet_2d_condition.UNet2DConditionModel'>.load_config(...) followed by <class 'unet_2d_condition.UNet2DConditionModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Initializing DINO model...


Using cache found in /home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main
/home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/zixiliu/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


Models initialized successfully!


## 3. Load Meshes

Load your source and target meshes. You can modify the paths below to use your own meshes.

In [9]:
# Mesh paths - modify these to use your own meshes
source_mesh_path = "meshes/oakink_beaker_decomp2.obj"
target_mesh_path = "meshes/oakink_mug_decomp.obj"

source_mesh = MeshContainer().load_from_file(source_mesh_path)
target_mesh = MeshContainer().load_from_file(target_mesh_path)


print(f"Source mesh: {len(source_mesh.vert)} vertices, {len(source_mesh.face)} faces")
print(f"Target mesh: {len(target_mesh.vert)} vertices, {len(target_mesh.face)} faces")

Source mesh: 11799 vertices, 23334 faces
Target mesh: 11392 vertices, 22696 faces


## 5. Configure Analysis Parameters

Set up the parameters for the similarity analysis.

In [10]:
# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 10  # Number of views for rendering (reduced for faster computation)

print(f"\nAnalysis Configuration:")
print(f"- Text prompt: '{prompt}'")
print(f"- Number of views: {num_views}")


Analysis Configuration:
- Text prompt: 'a beaker'
- Number of views: 10


# Multi point analysis

In [11]:
import numpy as np
source_points_3d = np.array([[-0.00793887, -0.0099004, -0.02071472], 
                             [0.01191107, 0.00449564, -0.02732792], 
                             [0.01248061, 0.00253191, -0.05360645], 
                             [0.01231679, 0.0029739, -0.00678797]])


# Analysis parameters
prompt = "a beaker"  # Text prompt for feature extraction
num_views = 1  # Number of views for rendering (reduced for faster computation)


# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.81s/it]


Number of missing features:  9575
Copied features from nearest vertices
Time taken in mins:  0.05395894050598145
Computing features for target mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.48s/it]

Number of missing features:  9447
Copied features from nearest vertices
Time taken in mins:  0.04745624462763468
  Point 0: Source vertex 7884 -> Target vertex 792 (similarity: 0.4473)
  Point 1: Source vertex 9922 -> Target vertex 57 (similarity: 0.5220)
  Point 2: Source vertex 197 -> Target vertex 0 (similarity: 0.3557)
  Point 3: Source vertex 10093 -> Target vertex 847 (similarity: 0.4324)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 792 (similarity: 0.4473)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 57 (similarity: 0.5220)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 0 (similarity: 0.3557)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 847 (similarity: 0.4324)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11392 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.4393
  - Min similarity: 0.3557
  - Max similarity: 0.5220
  - Total correspondences: 4

✅ Analysis complete!


In [12]:
num_views = 2  # Number of views for rendering (reduced for faster computation)


# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.62s/it]


Number of missing features:  5586
Copied features from nearest vertices
Time taken in mins:  0.1293342391649882
Computing features for target mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.52s/it]

Number of missing features:  5397
Copied features from nearest vertices
Time taken in mins:  0.09235415856043498
  Point 0: Source vertex 7884 -> Target vertex 792 (similarity: 0.4473)
  Point 1: Source vertex 9922 -> Target vertex 3234 (similarity: 0.7969)
  Point 2: Source vertex 197 -> Target vertex 21 (similarity: 0.3557)
  Point 3: Source vertex 10093 -> Target vertex 847 (similarity: 0.4324)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 792 (similarity: 0.4473)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 3234 (similarity: 0.7969)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 21 (similarity: 0.3557)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 847 (similarity: 0.4324)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11392 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.5081
  - Min similarity: 0.3557
  - Max similarity: 0.7969
  - Total correspondences: 4

✅ Analysis complete!


In [13]:
num_views = 3  # Number of views for rendering (reduced for faster computation)


# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.81s/it]


Number of missing features:  3356
Copied features from nearest vertices
Time taken in mins:  0.15901253620783487
Computing features for target mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.23s/it]

Number of missing features:  2813
Copied features from nearest vertices
Time taken in mins:  0.17385975122451783
  Point 0: Source vertex 7884 -> Target vertex 792 (similarity: 0.4473)
  Point 1: Source vertex 9922 -> Target vertex 3161 (similarity: 0.6377)
  Point 2: Source vertex 197 -> Target vertex 21 (similarity: 0.3557)
  Point 3: Source vertex 10093 -> Target vertex 3568 (similarity: 0.5601)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 792 (similarity: 0.4473)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 3161 (similarity: 0.6377)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 21 (similarity: 0.3557)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 3568 (similarity: 0.5601)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11392 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.5002
  - Min similarity: 0.3557
  - Max similarity: 0.6377
  - Total correspondences: 4

✅ Analysis complete!


In [14]:
num_views = 4  # Number of views for rendering (reduced for faster computation)


# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.85s/it]


Number of missing features:  2667
Copied features from nearest vertices
Time taken in mins:  0.20461530288060506
Computing features for target mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:10<00:00,  2.72s/it]

Number of missing features:  2353
Copied features from nearest vertices
Time taken in mins:  0.1990134914716085
  Point 0: Source vertex 7884 -> Target vertex 8776 (similarity: 0.7314)
  Point 1: Source vertex 9922 -> Target vertex 5488 (similarity: 0.5967)
  Point 2: Source vertex 197 -> Target vertex 21 (similarity: 0.3557)
  Point 3: Source vertex 10093 -> Target vertex 4884 (similarity: 0.4688)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 8776 (similarity: 0.7314)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 5488 (similarity: 0.5967)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 21 (similarity: 0.3557)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 4884 (similarity: 0.4688)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11392 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.5381
  - Min similarity: 0.3557
  - Max similarity: 0.7314
  - Total correspondences: 4

✅ Analysis complete!


In [15]:
num_views = 5  # Number of views for rendering (reduced for faster computation)


# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:15<00:00,  3.00s/it]


Number of missing features:  2326
Copied features from nearest vertices
Time taken in mins:  0.2725011507670085
Computing features for target mesh...
Rendering complete


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.87s/it]

Number of missing features:  2150
Copied features from nearest vertices
Time taken in mins:  0.25773532390594484
  Point 0: Source vertex 7884 -> Target vertex 8223 (similarity: 0.7344)
  Point 1: Source vertex 9922 -> Target vertex 3560 (similarity: 0.7212)
  Point 2: Source vertex 197 -> Target vertex 21 (similarity: 0.3557)
  Point 3: Source vertex 10093 -> Target vertex 3559 (similarity: 0.6772)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 8223 (similarity: 0.7344)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 3560 (similarity: 0.7212)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 21 (similarity: 0.3557)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 3559 (similarity: 0.6772)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11392 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.6221
  - Min similarity: 0.3557
  - Max similarity: 0.7344
  - Total correspondences: 4

✅ Analysis complete!


In [16]:
num_views = 10  # Number of views for rendering (reduced for faster computation)


# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:32<00:00,  3.23s/it]


Number of missing features:  982
Copied features from nearest vertices
Time taken in mins:  0.5795831918716431
Computing features for target mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:30<00:00,  3.02s/it]

Number of missing features:  1590
Copied features from nearest vertices
Time taken in mins:  0.5382819096247355
  Point 0: Source vertex 7884 -> Target vertex 9296 (similarity: 0.7969)
  Point 1: Source vertex 9922 -> Target vertex 3626 (similarity: 0.8384)
  Point 2: Source vertex 197 -> Target vertex 21 (similarity: 0.3557)
  Point 3: Source vertex 10093 -> Target vertex 3273 (similarity: 0.7959)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 9296 (similarity: 0.7969)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 3626 (similarity: 0.8384)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 21 (similarity: 0.3557)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 3273 (similarity: 0.7959)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11392 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.6967
  - Min similarity: 0.3557
  - Max similarity: 0.8384
  - Total correspondences: 4

✅ Analysis complete!


In [17]:
num_views = 15  # Number of views for rendering (reduced for faster computation)


# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:43<00:00,  2.87s/it]


Number of missing features:  577
Copied features from nearest vertices
Time taken in mins:  0.7689466516176859
Computing features for target mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:34<00:00,  2.29s/it]

Number of missing features:  1202
Copied features from nearest vertices
Time taken in mins:  0.614670459429423
  Point 0: Source vertex 7884 -> Target vertex 9291 (similarity: 0.7368)
  Point 1: Source vertex 9922 -> Target vertex 3605 (similarity: 0.7598)
  Point 2: Source vertex 197 -> Target vertex 21 (similarity: 0.3557)
  Point 3: Source vertex 10093 -> Target vertex 3235 (similarity: 0.7603)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 9291 (similarity: 0.7368)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 3605 (similarity: 0.7598)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 21 (similarity: 0.3557)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 3235 (similarity: 0.7603)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11392 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.6531
  - Min similarity: 0.3557
  - Max similarity: 0.7603
  - Total correspondences: 4

✅ Analysis complete!


In [18]:
num_views = 20  # Number of views for rendering (reduced for faster computation)


# Run the complete analysis pipeline
similarity_colors, raw_similarities, source_point_idx, closest_distance = run_multi_point_correspondence_analysis(
    source_mesh=source_mesh,
    target_mesh=target_mesh,
    source_points_3d=source_points_3d,
    device=device,
    pipe=pipe,
    dino_model=dino_model,
    prompt=prompt,
    num_views=num_views
)

print("\n✅ Analysis complete!")

Finding correspondences for 4 source points...
Closest vertices to input 3D points:
  Point 0: [-0.00793887 -0.0099004  -0.02071472] -> Vertex 7884 (distance: 0.0246)
  Point 1: [ 0.01191107  0.00449564 -0.02732792] -> Vertex 9922 (distance: 0.0217)
  Point 2: [ 0.01248061  0.00253191 -0.05360645] -> Vertex 197 (distance: 0.0112)
  Point 3: [ 0.01231679  0.0029739  -0.00678797] -> Vertex 10093 (distance: 0.0221)
Computing features for source mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:49<00:00,  2.47s/it]


Number of missing features:  521
Copied features from nearest vertices
Time taken in mins:  0.8800276041030883
Computing features for target mesh...
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:50<00:00,  2.53s/it]

Number of missing features:  1199
Copied features from nearest vertices
Time taken in mins:  0.8994696696599325
  Point 0: Source vertex 7884 -> Target vertex 9561 (similarity: 0.8096)
  Point 1: Source vertex 9922 -> Target vertex 5925 (similarity: 0.8276)
  Point 2: Source vertex 197 -> Target vertex 21 (similarity: 0.3557)
  Point 3: Source vertex 10093 -> Target vertex 5523 (similarity: 0.8291)
🎨 Visualizing 4 correspondences:
  Correspondence 1: Source vertex 7884 ↔ Target vertex 9561 (similarity: 0.8096)
    Color: RGB(1.00, 0.20, 0.20)
  Correspondence 2: Source vertex 9922 ↔ Target vertex 5925 (similarity: 0.8276)
    Color: RGB(0.45, 0.90, 0.00)
  Correspondence 3: Source vertex 197 ↔ Target vertex 21 (similarity: 0.3557)
    Color: RGB(0.20, 1.00, 1.00)
  Correspondence 4: Source vertex 10093 ↔ Target vertex 5523 (similarity: 0.8291)
    Color: RGB(0.45, 0.00, 0.90)
🎯 Creating combined mesh + 3D marker visualization...


✅ Combined mesh + 3D octahedron markers visualization created!
   Each correspondence pair has matching colored 3D octahedrons
   Source mesh: 11799 vertices + 24 marker vertices
   Target mesh: 11392 vertices + 24 marker vertices

📊 Correspondence Statistics:
  - Average similarity: 0.7055
  - Min similarity: 0.3557
  - Max similarity: 0.8291
  - Total correspondences: 4

✅ Analysis complete!
